# Chatbot para inmobiliaria utilizando GPT

Fecha: 28/04/2023

## Objetivo:

Crear un chatbot que permita responder preguntas a partir de un archivo en formato PDF. El archivo debe de tener texto que permita al chatbot responder preguntas realizadas por un usuario.

El chatbot estará entrenado con datos relacionados al rubro de inmobiliaria.

## Procedimiento:

Considerar la secuencia de pasos indicados.

Los archivos PDF deberán ubicarse en la carpeta o directorio `/dataset`. Debajo en el código aparece la línea de comando para crear esta carpeta.



## Referencias:

- Word embedding [https://youtu.be/-XVkdIdli0I](https://youtu.be/-XVkdIdli0I)

- Cómo Usar ChatGPT Sobre Información Interna De Tu Negocio [https://youtu.be/hVXpAh1FQCQ?t=725](https://youtu.be/hVXpAh1FQCQ?t=725)


## Paso 0: Conectar a Google Drive



In [1]:
# Conexión con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Paso 1: Configuración e instalación de librerías

### Instalar librerías

- langchain: Interactuar con LLMs
- llama-index: Chunks e indexar

In [2]:
!pip install PyPDF2 openai langchain llama-index==0.5.25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.0/782.0 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.8/548.8 kB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 k

### Importar librerías

In [3]:
import os
import pandas as pd
import openai
from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI
import textwrap

# Procesar datos de PDF
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

#import gradio as gr
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity

### Clases

#### Clase ChatBotInmobiliaria

In [4]:
class ChatBotInmobiliaria():
    def __init__(self):
        self.embedding_engine = "text-embedding-ada-002"
        self.model_name = "gpt-3.5-turbo"
        self.index = None
    
    def create_dataset(self, directory_path, filepath_dataset):
        # directory_path: Directorio donde se ubican los archivos PDF.
        # filepath_dataset: Nombre del archivo JSON vectorizado.
        if directory_path != None:
            #Leer los PDFs
            pdf = SimpleDirectoryReader(directory_path).load_data()
            #Definir e instanciar el modelo
            modelo = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name=self.model_name))
            #Indexar el contenido de los PDFs
            service_context = ServiceContext.from_defaults(llm_predictor=modelo)
            self.index = GPTSimpleVectorIndex.from_documents(pdf, service_context = service_context)
            self.__save_model(filepath_dataset)

    def __save_model(self, filepath):
        #Guardar el índice a disco para no tener que repetir cada vez
        #Recordar que necesistaríamos persistir el drive para que lo mantenga
        self.index.save_to_disk(filepath)
    
    def load_dataset(self, filepath):
        #Cargar el índice del disco
        self.index = GPTSimpleVectorIndex.load_from_disk(filepath)

    def ask(self, question=""):
        if len(question) == 0:
            print("Debe de ingresar una pregunta.")
        try:
            respuesta = self.index.query(question)
            for frase in textwrap.wrap(respuesta.response, width=100):
                print(frase)
        except Exception as e:
            print(e)

## Paso 2: Configurar variables

In [5]:
# Crear la carpeta datos y mover los archivos PDF a esa carpeta

dataset_directory = "/content/drive/MyDrive/dataset"

if os.path.exists(dataset_directory) == False:
    os.mkdir(dataset_directory)

In [6]:
# API de OpenAI
os.environ["OPENAI_API_KEY"] = "sk-"

DATASET_INMOBILIARIA_JSON = "demo-inmobiliaria.json"

## Paso 3: Procesamiento

In [7]:
chatbot = ChatBotInmobiliaria()

In [9]:
# CUIDADO !!
#
# Carga los datos del PDF y los guarda en un archivo JSON.
# Si ya existe el archivo JSON, obviar este paso.
#

# chatbot.create_dataset(dataset_directory, DATASET_INMOBILIARIA_JSON)

In [10]:
chatbot.load_dataset(DATASET_INMOBILIARIA_JSON)

### Testing

In [ ]:
# Ejemplos:
# ¿Cuánto es el precio promedio de una vivienda?
# ¿Qué regulaciones tengo para comprar una vivienda?
# Qué riesgos debo tener en cuenta para comprar una inmobiliaria?
# qué influencia europea hay en la compra de una vivienda?

while True:
    pregunta = input('Escribe tu pregunta   \n') + "Responde en español" 
    chatbot.ask(pregunta)

# Interfaz Gradio

## Instalar librerías

In [12]:
!pip install gradio plotly scikit-learn PyPDF2 openai langchain llama-index==0.5.25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3

## Interfaz

In [13]:
import json
import gradio as gr
from pathlib import Path
import os
import pandas as pd
import time
import random

import openai
from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext
from langchain.chat_models import ChatOpenAI
import textwrap

# Procesar datos de PDF
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

#import gradio as gr
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity


# API KEY OPENAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# CONSTANTES
DATASET_JSON = "demo-inmobiliaria.json"

# Ubicación dataset
carpeta_actual = os.getcwd()
print(f"Nombre de la carpeta actual: {carpeta_actual}")
PATH_FILE = f"{os.getcwd()}/{DATASET_JSON}"

class ChatBotInmobiliaria():
    def __init__(self):
        self.embedding_engine = "text-embedding-ada-002"
        self.model_name = "gpt-3.5-turbo"
        self.index = None
    
    def create_dataset(self, directory_path, filepath_dataset):
        # directory_path: Directorio donde se ubican los archivos PDF.
        # filepath_dataset: Nombre del archivo JSON vectorizado.
        if directory_path != None:
            #Leer los PDFs
            pdf = SimpleDirectoryReader(directory_path).load_data()
            #Definir e instanciar el modelo
            modelo = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name=self.model_name))
            #Indexar el contenido de los PDFs
            service_context = ServiceContext.from_defaults(llm_predictor=modelo)
            self.index = GPTSimpleVectorIndex.from_documents(pdf, service_context = service_context)
            self.__save_model(filepath_dataset)

    def __save_model(self, filepath):
        #Guardar el índice a disco para no tener que repetir cada vez
        #Recordar que necesistaríamos persistir el drive para que lo mantenga
        self.index.save_to_disk(filepath)
    
    def load_dataset(self, filepath):
        #Cargar el índice del disco
        self.index = GPTSimpleVectorIndex.load_from_disk(filepath)

    def ask(self, question=""):
        if len(question) == 0:
            print("Debe de ingresar una pregunta.")
        try:
            return self.index.query(question + "\nResponde en español")
        except Exception as e:
            print(e)
            return "Hubo un error."
    

# Gradio

description ="""
<p>
<center>
Demo Inmobiliaria, el objetivo es responder preguntas a través de OpenAI previamente entrenado con un archivo PDF.
<img src="https://raw.githubusercontent.com/All-Aideas/sea_apirest/main/logo.png" alt="logo" width="250"/>
</center>
</p>
"""

article = "<p style='text-align: center'><a href='http://allaideas.com/index.html' target='_blank'>Demo Inmobiliaria: Link para más info</a> </p>"
examples = [["¿Cuánto está una casa en San Isidro?"],["Hay precios más baratos?"],["A dónde llamo?", "Qué leyes existen?"]]

gpt_bot = ChatBotInmobiliaria()
gpt_bot.load_dataset(PATH_FILE)
chat_history = []

def chat(pregunta):
    bot_message = str(gpt_bot.ask(question=pregunta))
    chat_history.append((pregunta, bot_message))
    time.sleep(1)
    return chat_history

in1 = gr.inputs.Textbox(label="Pregunta")
out1 = gr.outputs.Chatbot(label="Respuesta").style(height=350)

demo = gr.Interface(
    fn=chat,
    inputs=in1,
    outputs=out1,
    title="Demo Inmobiliaria",
    description=description,
    article=article,
    enable_queue=True,
    examples=examples,
    )

demo.launch(debug=True)


Nombre de la carpeta actual: /content


/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:309: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/deprecation.py:40: UserWarning: `enable_queue` is deprecated in `Interface()`, please use it within `launch()` instead.
  warnings.warn(value)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
